In [239]:
import numpy as np
from enum import Enum

class ActivationFuction(Enum):
    SIGMOID = 1
    RELU = 2,
    ELU = 3 # a = 1

class Problem(Enum):
    Classification = 1,
    Regression = 2

class NeuralNetwork:
    def __init__(self, 
                 input_size = 784, 
                 hidden_size = 128, 
                 output_size = 10, 
                 hidden_layers_count = 1, 
                 inner_activation_function = ActivationFuction.SIGMOID,
                 problem = Problem.Classification,
                 learning_rate = 0.01, 
                 num_epochs = 10, 
                 seed = 0, 
                 use_bias = True) -> None:
        self.seed = seed
        np.random.seed(self.seed)
        self.use_bias = use_bias
        self.hidden_layers_count = hidden_layers_count
        self.input_size = input_size 
        self.hidden_size = hidden_size
        
        if problem == Problem.Regression:
            self.output_size = 1
            self.input_size = 1
        else:
            self.output_size = output_size
            
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.inner_activation_function = inner_activation_function
        self.problem = problem
 
        self.layers_sizes = [self.input_size] + [self.hidden_size] * self.hidden_layers_count + [self.output_size]
        self.layers_count = len(self.layers_sizes)
        self.weights_count = self.layers_count - 1

        self.weights = [None] * self.weights_count 
        self.bias = [None] * self.weights_count 
        self.create_weights_array()

    def create_weights_array(self):
        for i in range(self.weights_count):
            if self.inner_activation_function == ActivationFuction.SIGMOID:
                self.weights[i] = np.random.uniform(-np.sqrt(6 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), np.sqrt(6 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), size=(self.layers_sizes[i], self.layers_sizes[i + 1]))
            else:
                self.weights[i] = np.random.normal(0, np.sqrt(2 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), size=(self.layers_sizes[i], self.layers_sizes[i + 1]))
    
            if self.use_bias:
                self.bias[i] = np.zeros(self.layers_sizes[i + 1])
                
    def train_and_check_acc(self, train_inputs, train_results, test_inputs, test_results):
    
        for epoch in range(self.num_epochs):
            for i in range(len(train_inputs)):
                self.train(train_inputs[i], train_results[i])
                
            correct = 0
            nn_results = []
            for i in range(len(test_inputs)):
                values = self.predict(test_inputs[i])
                nn_results.append(values[-1])
                
                if (self.problem == Problem.Classification):  
                   if np.argmax(values[-1]) == np.argmax(test_results[i]):
                        correct += 1
                        
            error = self.accuracy(nn_results, test_results)  
            if (self.problem == Problem.Classification):
                test_accuracy = correct / len(test_inputs)
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Test Accuracy: {test_accuracy * 100:.2f}% Loss: {error}, Correct: {correct}, All: {len(test_inputs)} ")
            else:
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Test Accuracy: {error}")
            
    def accuracy(self, results, expected_result):
        if self.problem == Problem.Classification:
            return -np.sum(expected_result * np.log(results[-1]))
        else:
            return np.average(np.power(expected_result - results, 2))

        
    def train(self, input, expected_result): # Backpropagation
        values = self.predict(input)
    
        delta = [None] * self.weights_count
        delta[-1] = values[-1] - expected_result
        
        for weightId in range(self.weights_count - 2, -1, -1):
            delta[weightId] = np.dot(delta[weightId + 1], self.weights[weightId + 1].T) * self.neuron_activation_derivative(values[weightId + 1])
            
        for weightId in range(self.weights_count):
            self.weights[weightId] -= np.outer(values[weightId], delta[weightId]) * self.learning_rate 
            if self.use_bias:
                self.bias[weightId] = self.bias[weightId] - delta[weightId] * self.learning_rate
        
    
    def neuron_activation(self, v):
        if self.inner_activation_function == ActivationFuction.SIGMOID:
            return 1 / (1 + np.exp(-v))
        elif self.inner_activation_function == ActivationFuction.RELU:
            return np.maximum(0, v)
        elif self.inner_activation_function == ActivationFuction.ELU:
            return (v > 0) * 1 + (v <= 0) * np.exp(v) 
        
    def neuron_activation_derivative(self, v):
        if self.inner_activation_function == ActivationFuction.SIGMOID:
            return v * (1 - v)
        elif self.inner_activation_function == ActivationFuction.RELU:
            return (v > 0) * 1
        elif self.inner_activation_function == ActivationFuction.ELU:
            return (v > 0) * v + (v <= 0) * (np.exp(v) - 1) 
    
    def output_activation(self, x):
        if self.problem == Problem.Classification:
            exp_x = np.exp(x - np.max(x))
            return exp_x / exp_x.sum()
        else: 
            return x
    
    def predict(self, input): #forward propagation
        values = [None] * self.layers_count
        
        values[0] = input
        
        for weightId in range(self.weights_count):
           v = np.dot(values[weightId], self.weights[weightId]) + self.bias[weightId] if self.use_bias else 0
           if weightId != self.weights_count - 1:
               v = self.neuron_activation(v)
           else:    
               v = self.output_activation(v)
               
           values[weightId + 1] = v  
        
        return values

In [223]:
import numpy as np
from mnist.loader import MNIST

# Load the MNIST dataset
mndata = MNIST("C:\\Users\\muzyk\\Downloads\\mnist")  # Replace with the path to your MNIST data
mndata.gz = True

# Load the training and testing data
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

# Convert to NumPy arrays for easier manipulation
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

num_classes = 10
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

nn = NeuralNetwork(inner_activation_function = ActivationFuction.SIGMOID)

nn.train_and_check_acc(train_images, train_labels, test_images, test_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Epoch 1/10, Test Accuracy: 92.39% Loss: 109838.40205957575, Correct: 9239, All: 10000 
Epoch 2/10, Test Accuracy: 94.60% Loss: 105290.46652658422, Correct: 9460, All: 10000 


KeyboardInterrupt: 

In [240]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.simple.test.100.csv", sep=",")
train = pd.read_csv("classification/data.simple.train.100.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 2
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 8, hidden_layers_count = 1, num_epochs = 100, inner_activation_function = ActivationFuction.SIGMOID)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

Epoch 1/100, Test Accuracy: 47.00% Loss: 71.8848281868098, Correct: 47, All: 100 
Epoch 2/100, Test Accuracy: 26.00% Loss: 69.36252357913196, Correct: 26, All: 100 
Epoch 3/100, Test Accuracy: 37.00% Loss: 69.27108505166932, Correct: 37, All: 100 
Epoch 4/100, Test Accuracy: 58.00% Loss: 69.37767267644239, Correct: 58, All: 100 
Epoch 5/100, Test Accuracy: 79.00% Loss: 69.58103275522006, Correct: 79, All: 100 
Epoch 6/100, Test Accuracy: 87.00% Loss: 69.87161060329402, Correct: 87, All: 100 
Epoch 7/100, Test Accuracy: 93.00% Loss: 70.25529210047841, Correct: 93, All: 100 
Epoch 8/100, Test Accuracy: 96.00% Loss: 70.74606537943043, Correct: 96, All: 100 
Epoch 9/100, Test Accuracy: 98.00% Loss: 71.36254813836366, Correct: 98, All: 100 
Epoch 10/100, Test Accuracy: 99.00% Loss: 72.12595343358063, Correct: 99, All: 100 
Epoch 11/100, Test Accuracy: 99.00% Loss: 73.05849425231557, Correct: 99, All: 100 
Epoch 12/100, Test Accuracy: 99.00% Loss: 74.18180069852662, Correct: 99, All: 100 
Ep

In [224]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.simple.test.10000.csv", sep=",")
train = pd.read_csv("classification/data.simple.train.10000.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 2
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 8, hidden_layers_count = 2, num_epochs = 20, inner_activation_function = ActivationFuction.SIGMOID)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

Epoch 1/20, Test Accuracy: 99.29% Loss: 13144.543980401108, Correct: 9929, All: 10000 
Epoch 2/20, Test Accuracy: 99.62% Loss: 23370.381661655236, Correct: 9962, All: 10000 
Epoch 3/20, Test Accuracy: 99.61% Loss: 30042.126455795216, Correct: 9961, All: 10000 
Epoch 4/20, Test Accuracy: 99.64% Loss: 34948.087385028855, Correct: 9964, All: 10000 
Epoch 5/20, Test Accuracy: 99.63% Loss: 38843.42699606973, Correct: 9963, All: 10000 
Epoch 6/20, Test Accuracy: 99.65% Loss: 42084.93332066136, Correct: 9965, All: 10000 
Epoch 7/20, Test Accuracy: 99.65% Loss: 44867.11396571289, Correct: 9965, All: 10000 
Epoch 8/20, Test Accuracy: 99.66% Loss: 47307.489343299065, Correct: 9966, All: 10000 
Epoch 9/20, Test Accuracy: 99.65% Loss: 49482.56977422134, Correct: 9965, All: 10000 
Epoch 10/20, Test Accuracy: 99.65% Loss: 51445.05000707353, Correct: 9965, All: 10000 
Epoch 11/20, Test Accuracy: 99.64% Loss: 53232.84779439861, Correct: 9964, All: 10000 
Epoch 12/20, Test Accuracy: 99.63% Loss: 54874.

In [225]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.three_gauss.test.10000.csv", sep=",")
train = pd.read_csv("classification/data.three_gauss.train.10000.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 3
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 6, hidden_layers_count = 1, num_epochs = 50, inner_activation_function = ActivationFuction.SIGMOID)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

Epoch 1/50, Test Accuracy: 33.33% Loss: 133827.55862143144, Correct: 10000, All: 30000 
Epoch 2/50, Test Accuracy: 33.33% Loss: 136177.99542176604, Correct: 10000, All: 30000 
Epoch 3/50, Test Accuracy: 33.33% Loss: 141890.10915416034, Correct: 10000, All: 30000 
Epoch 4/50, Test Accuracy: 33.33% Loss: 150685.89612457744, Correct: 10000, All: 30000 
Epoch 5/50, Test Accuracy: 33.33% Loss: 159864.30123881265, Correct: 10000, All: 30000 
Epoch 6/50, Test Accuracy: 33.33% Loss: 168111.72302442804, Correct: 10000, All: 30000 
Epoch 7/50, Test Accuracy: 33.35% Loss: 175295.69288388942, Correct: 10004, All: 30000 
Epoch 8/50, Test Accuracy: 34.01% Loss: 181541.59685305684, Correct: 10202, All: 30000 
Epoch 9/50, Test Accuracy: 35.49% Loss: 187018.0768503781, Correct: 10646, All: 30000 
Epoch 10/50, Test Accuracy: 36.90% Loss: 191906.51952974027, Correct: 11071, All: 30000 
Epoch 11/50, Test Accuracy: 38.12% Loss: 196366.13911099656, Correct: 11435, All: 30000 
Epoch 12/50, Test Accuracy: 39.

In [226]:
import numpy as np
import pandas as pd

test = pd.read_csv("regression/data.cube.test.100.csv", sep=",")
train = pd.read_csv("regression/data.cube.train.100.csv", sep=",")

train_vectors = np.array(train["x"])
train_results = np.array(train["y"])
test_vectors = np.array(test["x"])
test_results = np.array(test["y"])

nn = NeuralNetwork(hidden_size = 6, hidden_layers_count = 1, num_epochs = 20, inner_activation_function = ActivationFuction.RELU, problem = Problem.Regression)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

Epoch 1/20, Test Accuracy: 134243.47854882723
Epoch 2/20, Test Accuracy: 133524.77838048464
Epoch 3/20, Test Accuracy: 133266.78702831615
Epoch 4/20, Test Accuracy: 133173.03395099868
Epoch 5/20, Test Accuracy: 133138.80841236367
Epoch 6/20, Test Accuracy: 133126.2929665212
Epoch 7/20, Test Accuracy: 133121.71354422753
Epoch 8/20, Test Accuracy: 133120.03754670746
Epoch 9/20, Test Accuracy: 133119.42410677206
Epoch 10/20, Test Accuracy: 133119.19957185013
Epoch 11/20, Test Accuracy: 133119.11738533387
Epoch 12/20, Test Accuracy: 133119.08730248164
Epoch 13/20, Test Accuracy: 133119.07629119421
Epoch 14/20, Test Accuracy: 133119.0722607082
Epoch 15/20, Test Accuracy: 133119.07078542002
Epoch 16/20, Test Accuracy: 133119.07024541692
Epoch 17/20, Test Accuracy: 133119.0700477583
Epoch 18/20, Test Accuracy: 133119.06997540893
Epoch 19/20, Test Accuracy: 133119.0699489267
Epoch 20/20, Test Accuracy: 133119.0699392333
